In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
names = pd.read_csv('data/dapps_names_2020-12-09.csv')
names.shape

In [ ]:
df = pd.read_csv('data/dapps_with_licences_2020-12-09.csv')
df.shape

In [ ]:
print('There are', names.slug.nunique() - df.slug.nunique(), 'Dapps that do not reveal their smart contract hashes at all')

In [ ]:
df.created = pd.to_datetime(df.created) 
contracts_per_timepoint = pd.DataFrame(df.resample('M', on='created').count()['slug'])
contracts_per_year = pd.DataFrame(df.resample('Y', on='created').count()['slug'])

In [ ]:
df_grouped = df.groupby('slug', as_index=False)['hash'].count()
df_grouped = df_grouped.merge(df[['slug', 'created']], on='slug')
df_grouped.drop_duplicates(inplace=True)
#df.groupby(by=[df.created.dt.month, df.created.dt.year]).mean()

In [ ]:
monthly_mean = df_grouped.set_index('created').resample('1M').mean()
yearly_mean = df_grouped.set_index('created').resample('1Y').mean()

In [ ]:
without_sc = names[names.slug.isin(set(names.slug) - set(df.slug))]
without_sc.created = pd.to_datetime(without_sc.created)
without_sc_grouped_year = without_sc.resample('M', on='created').count().drop(['created', 'name', 'rank', 'slug', 'teaser'], axis=1)
without_sc_grouped_year = without_sc_grouped_year.join(monthly_mean, how='left')
without_sc_grouped_year.fillna(0, inplace=True)

In [ ]:
sc_from_dapps = pd.DataFrame(contracts_per_timepoint['slug'] + without_sc_grouped_year['hash'])
sc_from_dapps.to_csv('data/source_codes_from_dapps.csv')